In [ ]:
!pip install -U bitsandbytes
!pip install -U transformers
!pip install -U peft
!pip install -U accelerate
!pip install -U datasets
!pip install -U trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch
from datasets import load_dataset
from trl import SFTTrainer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# base_model = "microsoft/phi-2"
base_model = "mistralai/Mistral-7B-v0.1"
# dataset_name = "hieunguyenminh/roleplay"
dataset_name = "tatsu-lab/alpaca"
# new_model = "phi-2-finetunedTextGeneration"
new_model = "mistral-finetunedTextGeneration"

In [ ]:
#Importing the dataset
dataset = load_dataset(dataset_name, split="train[0:30000]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset["text"][200]

'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat geometric shape has 5 sides and 5 angles?\n\n### Response:\nThe geometric shape is a pentagon.'

In [ ]:
print(len(dataset))

30000


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model.config.use_cache = False
model.config.pretraining_tp = 1


# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense',
        'fc1',
        'fc2',
    ]
)
model = get_peft_model(model, peft_config)
model.get_memory_footprint()

5113929728

In [ ]:
training_arguments = TrainingArguments(
    output_dir="/content/drive/My Drive/Assignment1C-NLP/Mistral/FromTrainArg",
    num_train_epochs=1,
    per_device_train_batch_size=20,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_strategy="steps",
    save_steps=1000,
    logging_steps=500,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    disable_tqdm=False,
    report_to="none",
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= 2048,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)


Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
500,0.830100
1000,0.792100
1500,0.781000


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=1500, training_loss=0.8010681355794271, metrics={'train_runtime': 9884.4893, 'train_samples_per_second': 3.035, 'train_steps_per_second': 0.152, 'total_flos': 1.645605427470336e+17, 'train_loss': 0.8010681355794271, 'epoch': 1.0})

In [ ]:
trainer.save_model("/content/drive/My Drive/Assignment1C-NLP/Mistral/FromSave")
trainer.save_model(new_model)
trainer.tokenizer.save_pretrained(new_model)

('mistral-finetunedTextGeneration/tokenizer_config.json',
 'mistral-finetunedTextGeneration/special_tokens_map.json',
 'mistral-finetunedTextGeneration/tokenizer.model',
 'mistral-finetunedTextGeneration/added_tokens.json',
 'mistral-finetunedTextGeneration/tokenizer.json')

In [ ]:
prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: What is the capital of Nepal?"
pipe = pipeline(task="text-generation",
                model=base_model,
                tokenizer=tokenizer,
                max_new_tokens=100)
result = pipe(f"{prompt}")
print(result[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: What is the capital of Nepal? ### Response: Kathmandu ### Instruction: What is the capital of Nepal? ### Response: Kathmandu ### Instruction: What is the capital of Nepal? ### Response: Kathmandu ### Instruction: What is the capital of Nepal? ### Response: Kathmandu ### Instruction: What is the capital of Nepal? ### Response: Kathmandu ### Instruction: What is the capital of Nepal? ### Response: Kathmandu ### Instruction:


In [ ]:
prompt1 = "Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: What is the capital of Nepal?"
pipe1 = pipeline(task="text-generation",
                model=new_model,
                tokenizer=new_model,
                max_new_tokens=100)
result1 = pipe1(f"{prompt1}")
print(result1[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: What is the capital of Nepal? ### Response: Kathmandu ### Instruction: What is the capital of Nepal? ### Response: Kathmandu ### Instruction: What is the capital of Nepal? ### Response: Kathmandu ### Instruction: What is the capital of Nepal? ### Response: Kathmandu ### Instruction: What is the capital of Nepal? ### Response: Kathmandu ### Instruction: What is the capital of Nepal? ### Response: Kathmandu ### Instruction:
